# High performance pandas

```py
mask = (x > .5) & (y < .5)
df[df[mask]]
```

under the hood 

```py
tmp1 = (x > .5)
tmp2 = (y < .5)
mask = tmp1 & tmp2
```

use pd.eval(""), df.query() -> does elementwise operation with numexpr

In [2]:
import numpy as np 
import pandas as pd 

nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.119810,1.756696,1.027536,1.900217,-0.988676,0.099452,0.388992,-0.136126,1.874327,-1.703779,...,-0.475074,0.693387,0.667072,-0.623931,-0.905582,1.673389,0.053968,0.037077,0.644471,0.053314
1,0.668287,-0.284818,0.293121,1.370264,0.331326,-0.641949,2.009645,-0.243579,-1.093921,1.285410,...,-0.434888,-0.028926,0.754711,-0.440591,1.948920,-0.162380,-0.361521,-1.108245,-0.296122,-2.269667
2,0.223424,0.901689,1.209006,-0.348657,0.030399,-0.162859,0.155839,-0.126578,-0.897469,-2.060019,...,-0.966768,0.082628,-0.159249,0.870168,-0.760554,-0.164833,0.283951,-1.441617,-0.912886,-0.136703
3,0.651442,0.534086,-0.430196,0.640432,1.803624,0.124135,0.198049,-1.943168,-0.144802,-0.748018,...,-1.083782,-0.815533,1.656511,0.527475,-1.628603,0.589315,-0.853152,0.692521,1.373794,-1.379616
4,-1.069791,1.772242,1.421139,-0.211154,-2.127580,1.064145,-1.237648,-1.486588,-0.498436,-0.009166,...,0.110317,0.816555,0.547072,0.596486,2.871553,-1.324869,1.001328,0.812029,1.251172,0.230592


In [4]:
df1.shape

(1000000, 100)

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [7]:
%timeit df1+df2+df3+df4
%timeit pd.eval("df1+df2+df3+df4")

748 ms ± 7.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
310 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

sum_eval.equals(standard)

True

In [9]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,3,1,4
1,3,4,5
2,1,4,5
3,1,3,1
4,4,3,4
5,4,5,2


In [12]:
rolls.eval("Sum = Die1 + Die2 + Die3", inplace=True)
rolls

,Die1,Die2,Die3,Sum
0,3,1,4,8
1,3,4,5,12
2,1,4,5,10
3,1,3,1,5
4,4,3,4,11
5,4,5,2,11


In [14]:
high = 9
rolls.eval("Winner = Sum > @high") 

,Die1,Die2,Die3,Sum,Winner
0,3,1,4,8,False
1,3,4,5,12,True
2,1,4,5,10,True
3,1,3,1,5,False
4,4,3,4,11,True
5,4,5,2,11,True


In [15]:
# traditional way to filter from a dataframe
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum
1,3,4,5,12
2,1,4,5,10
4,4,3,4,11
5,4,5,2,11


## Query

- filter using query

In [18]:
rolls.query("Sum > @high & Die1 == 3")

,Die1,Die2,Die3,Sum
1,3,4,5,12


In [21]:
#df_os = pd.read_csv("../Data/athlete_events.csv")
df_os = pd.read_csv("https://raw.githubusercontent.com/kokchun/Databehandling-AI22/main/Data/athlete_events.csv")

df_os.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [24]:
df_os[df_os["NOC"] == "SWE"].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [26]:
df_os.query("NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
725,414,Arvid berg,M,26.0,NaN,NaN,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Hammer Throw,NaN
726,415,Bjrn Olof Conny berg,M,23.0,181.0,76.0,Sweden,SWE,1992 Winter,1992,Winter,Albertville,Freestyle Skiing,Freestyle Skiing Men's Moguls,NaN
727,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Long Jump,Bronze
728,416,Nils Georg berg,M,19.0,181.0,78.0,Sweden,SWE,1912 Summer,1912,Summer,Stockholm,Athletics,Athletics Men's Triple Jump,Silver
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN


In [27]:
%timeit df_os[df_os["NOC"] == "SWE"]
%timeit df_os.query("NOC == 'SWE'")

13.8 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.2 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%timeit df_os[df_os["Height"] > 180]
%timeit df_os.query("Height > 180")

10.3 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.7 ms ± 519 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")].head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [33]:
df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'").head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN


In [34]:
%timeit df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")]
%timeit df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

24.4 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.6 ms ± 492 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
